This is 'model B0'. 'The base model' that is inspired from the state of the are model FgSegNet_S. 
Source: https://github.com/lim-anggun/FgSegNet

In [4]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8 MB 26 kB/s 
     |████████████████████████████████| 3.8 MB 53.2 MB/s 
     |████████████████████████████████| 448 kB 68.7 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=89b506cdeccf739aec72f8762354059fe6101e9c47f34c60fc7e1e6514d2b471
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [6]:
pip install keras==2.3.0

     |████████████████████████████████| 377 kB 5.0 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0


In [7]:
pip install -U scikit-learn==0.24

     |████████████████████████████████| 22.3 MB 827 kB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import time

In [13]:
import keras
from keras.models import Model
from keras.layers import Activation, Input, Dropout, BatchNormalization, SpatialDropout2D
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
from keras import regularizers
import time

#from my_upsampling_2d import MyUpSampling2D
import keras.backend as K
import tensorflow as tf

def loss(y_true, y_pred):
    void_label = -1.
    y_pred = K.reshape(y_pred, [-1])
    y_true = K.reshape(y_true, [-1])
    idx = tf.where(tf.not_equal(y_true, tf.constant(void_label, dtype=tf.float32)))
    y_pred = tf.gather_nd(y_pred, idx) 
    y_true = tf.gather_nd(y_true, idx)
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

def acc(y_true, y_pred):
    void_label = -1.
    y_pred = tf.reshape(y_pred, [-1])
    y_true = tf.reshape(y_true, [-1])
    idx = tf.where(tf.not_equal(y_true, tf.constant(void_label, dtype=tf.float32)))
    y_pred = tf.gather_nd(y_pred, idx) 
    y_true = tf.gather_nd(y_true, idx)
    return K.mean(K.equal(y_true, K.round(y_pred)), axis=-1)

def custom_f1(y_true, y_pred):    
    void_label = -1.
    y_pred = tf.reshape(y_pred, [-1])
    y_true = tf.reshape(y_true, [-1])
    idx = tf.where(tf.not_equal(y_true, tf.constant(void_label, dtype=tf.float32)))
    y_pred = tf.gather_nd(y_pred, idx) 
    y_true = tf.gather_nd(y_true, idx)
    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def loss2(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

def acc2(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)), axis=-1)


class FgSegNet_M_S_module(object):
    
    def __init__(self, lr, reg, img_shape, scene, vgg_weights_path):
        self.lr = lr
        self.reg = reg
        self.img_shape = img_shape
        self.scene = scene
        self.vgg_weights_path = vgg_weights_path

    def VGG16(self, x): 
        
        # Block 1
        x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format='channels_last')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    
        # Block 2
        x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
        x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
        # Block 3
        x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
        x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
        x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    
        # Block 4
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
        x = Dropout(0.5, name='dr1')(x)
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
        x = Dropout(0.5, name='dr2')(x)
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
        x = Dropout(0.5, name='dr3')(x)
        
        return x
        
    def transposedConv(self, x):
        
        # block 5
        x = Conv2DTranspose(64, (1, 1), activation='relu', padding='same', name='block5_tconv1', 
                                                kernel_regularizer=regularizers.l2(self.reg))(x)
        x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', name='block5_tconv2')(x)
        x = Conv2DTranspose(512, (1, 1), activation='relu', padding='same', name='block5_tconv3')(x)
        
        # block 6
        x = Conv2DTranspose(64, (1, 1), activation='relu', padding='same', name='block6_tconv1', 
                                                kernel_regularizer=regularizers.l2(self.reg))(x)
        x = Conv2DTranspose(64, (5, 5), strides=(2, 2), activation='relu', padding='same', name='block6_tconv2')(x)
        x = Conv2DTranspose(256, (1, 1), activation='relu', padding='same', name='block6_tconv3')(x)
        
        # block 7
        x = Conv2DTranspose(64, (1, 1), activation='relu', padding='same', name='block7_tconv1', 
                                                kernel_regularizer=regularizers.l2(self.reg))(x)
        x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', name='block7_tconv2')(x)
        x = Conv2DTranspose(128, (1, 1), activation='relu', padding='same', name='block7_tconv3')(x)
        
        # block 8
        x = Conv2DTranspose(64, (5, 5), strides=(2, 2), activation='relu', padding='same', name='block8_conv1', 
                                                kernel_regularizer=regularizers.l2(self.reg))(x)
        
        # block 9
        x = Conv2DTranspose(1, (1, 1), padding='same', name='block9_conv1')(x)
        x = Activation('sigmoid')(x)
        
        return x

    def initModel_M(self, dataset_name):
        assert dataset_name in ['CDnet', 'SBI', 'UCSD'], 'dataset_name must be either one in ["CDnet", "SBI", "UCSD"]]'
        assert len(self.img_shape)==3
        h, w, d = self.img_shape
        
        input_1 = Input(shape=(h, w, d), name='ip_scale1')
        vgg_layer_output = self.VGG16(input_1)
        shared_model = Model(inputs=input_1, outputs=vgg_layer_output, name='shared_model')
        shared_model.load_weights(self.vgg_weights_path, by_name=True)
        
        unfreeze_layers = ['block4_conv1','block4_conv2', 'block4_conv3']
        for layer in shared_model.layers:
            if(layer.name not in unfreeze_layers):
                layer.trainable = False
                
                
        # Scale 1
        x1 = shared_model.output
        # Scale 2
        input_2 = Input(shape=(int(h/2), int(w/2), d), name='ip_scale2')
        x2 = shared_model(input_2)
        x2 = UpSampling2D((2,2))(x2)
        # Scale 3
        input_3 = Input(shape=(int(h/4), int(w/4), d), name='ip_scale3')
        x3 = shared_model(input_3)
        x3 = UpSampling2D((4,4))(x3)
        
        
        if dataset_name=='CDnet':
            # Scale 1
            x1_ups = {'streetCornerAtNight':(0,1), 'tramStation':(1,0), 'turbulence2':(1,0)}
            if(self.scene=='wetSnow'):
                x1 = Cropping2D(cropping=((1, 2),(0, 0)))(x1)
            elif(self.scene=='skating'):
                x1 = Cropping2D(cropping=((1, 1),(1, 2)))(x1)
            else:
                for key, val in x1_ups.items():
                    if self.scene==key:
                        # upscale by adding number of pixels to each dim.
                        x1 = MyUpSampling2D(size=(1,1), num_pixels=val)(x1)
                        break
            
            # Scale 2
            x2_ups = {'tunnelExit_0_35fps':(0,1),'tramCrossroad_1fps':(1,0),'bridgeEntry':(1,1),
                      'busyBoulvard':(1,0),'fluidHighway':(0,1),'streetCornerAtNight':(1,1), 
                      'tramStation':(2,0),'winterStreet':(1,0),'twoPositionPTZCam':(1,0),
                      'peopleInShade':(1,1),'turbulence2':(1,1),'turbulence3':(1,0),
                      'skating':(1,1), 'wetSnow':(0,0)}
            for key, val in x2_ups.items():
                if self.scene == key and self.scene in ['skating', 'wetSnow']:
                    x2 = Cropping2D(cropping=((1, 1), val))(x2)
                    break
                elif self.scene==key:
                    x2 = MyUpSampling2D(size=(1, 1), num_pixels=val)(x2)
                    break
            
            # Scale 3
            x3_ups = {'tunnelExit_0_35fps':(2,3),'tramCrossroad_1fps':(3,0),'bridgeEntry':(3,1,),
                      'busyBoulvard':(3,0),'fluidHighway':(0,3),'streetCornerAtNight':(1,1),
                      'tramStation':(2,0),'winterStreet':(1,0),'twoPositionPTZCam':(1,2),
                      'peopleInShade':(1,3),'turbulence2':(3,1),'turbulence3':(1,0),
                      'office':(0,2), 'pedestrians':(0,2), 'bungalows':(0,2), 'busStation':(0,2)}
            
            for key, val in x3_ups.items():
                if self.scene==key:
                    x3 = MyUpSampling2D(size=(1,1), num_pixels=val)(x3)
                    break
                
        elif dataset_name=='SBI':
            if(self.scene=='Board'):
                x2 = MyUpSampling2D(size=(1,1), num_pixels=(1,0))(x2)
                x3 = MyUpSampling2D(size=(1,1), num_pixels=(1,2))(x3)
            elif(self.scene=='CaVignal'):
                x3 = MyUpSampling2D(size=(1,1), num_pixels=(2,2))(x3)
            elif(self.scene=='Foliage'):
                x3 = MyUpSampling2D(size=(1,1), num_pixels=(0,2))(x3)
            elif(self.scene=='Toscana'):
                x3 = MyUpSampling2D(size=(1,1), num_pixels=(2,0))(x3)
                
        elif dataset_name=='UCSD':
            x2_ups = {'birds':(1,0),'chopper':(1,0),'flock':(1,0),'freeway':(1,1),
                      'hockey':(1,1),'jump':(1,0),'landing':(1,1),'ocean':(1,1),
                      'rain':(1,1),'skiing':(1,0),'surf':(1,0),'traffic':(1,1),'zodiac':(1,1)}
            x3_ups = {'birds':(3,0),'boats':(0,2),'chopper':(3,0),'cyclists':(2,0),
                      'flock':(3,0),'freeway':(3,3),'hockey':(3,1),'jump':(3,0),
                      'landing':(3,1),'ocean':(1,3),'peds':(2,2),'rain':(1,1),
                      'skiing':(3,0),'surf':(3,0),'surfers':(0,2),'traffic':(1,1),'zodiac':(1,1)}
            
            for key, val in x2_ups.items():
                if self.scene==key:
                    x2 = MyUpSampling2D(size=(1,1), num_pixels=val)(x2)
                    break
                
            for key, val in x3_ups.items():
                if self.scene==key:
                    x3 = MyUpSampling2D(size=(1,1), num_pixels=val)(x3)
                    break
            
        # concatenate feature maps
        top = keras.layers.concatenate([x1, x2, x3], name='feature_concat')
        
        if dataset_name=='CDnet':
            if(self.scene=='wetSnow'):
                top = MyUpSampling2D(size=(1,1), num_pixels=(3,0))(top)
            elif(self.scene=='skating'):
                top = MyUpSampling2D(size=(1,1), num_pixels=(2,3))(top)
        
        # Transposed Conv
        top = self.transposedConv(top)
        
        if dataset_name=='CDnet':
            if(self.scene=='tramCrossroad_1fps'):
                top = MyUpSampling2D(size=(1,1), num_pixels=(2,0))(top)
            elif(self.scene=='bridgeEntry'):
                top = MyUpSampling2D(size=(1,1), num_pixels=(2,2))(top)
            elif(self.scene=='fluidHighway'):
                top = MyUpSampling2D(size=(1,1), num_pixels=(2,0))(top)
            elif(self.scene=='streetCornerAtNight'): 
                top = MyUpSampling2D(size=(1,1), num_pixels=(1,0))(top)
                top = Cropping2D(cropping=((0, 0),(0, 1)))(top)
            elif(self.scene=='tramStation'):  
                top = Cropping2D(cropping=((1, 0),(0, 0)))(top)
            elif(self.scene=='twoPositionPTZCam'):
                top = MyUpSampling2D(size=(1,1), num_pixels=(0,2))(top)
            elif(self.scene=='turbulence2'):
                top = Cropping2D(cropping=((1, 0),(0, 0)))(top)
                top = MyUpSampling2D(size=(1,1), num_pixels=(0,1))(top)
            elif(self.scene=='turbulence3'):
                top = MyUpSampling2D(size=(1,1), num_pixels=(2,0))(top)

        vision_model = Model(inputs=[input_1, input_2, input_3], outputs=top, name='vision_model')
        opt = keras.optimizers.RMSprop(lr = self.lr, rho=0.9, epsilon=1e-08, decay=0.0)
        
        # Since UCSD has no void label, we do not need to filter out
        if dataset_name == 'UCSD':
            c_loss = loss2
            c_acc = acc2
        else:
            c_loss = loss
            c_custom_f1 = custom_f1
            
        vision_model.compile(loss=c_loss, optimizer=opt, metrics=[c_custom_f1])
        return vision_model
    
    ### FgSegNet_S
    
    def FPM(self, x):
        x1 = MaxPooling2D((2, 2), strides=(1,1), padding='same')(x)
        x1 = Conv2D(64, (1, 1), padding='same')(x1)
        
        x2 = Conv2D(64, (3, 3), padding='same')(x)
        
        x3 = Conv2D(64, (3, 3), padding='same', dilation_rate=4)(x)
        
        x4 = Conv2D(64, (3, 3), padding='same', dilation_rate=8)(x)
        
        x5 = Conv2D(64, (3, 3), padding='same', dilation_rate=16)(x)
        
        x = keras.layers.concatenate([x1, x2, x3, x4, x5], axis=-1)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SpatialDropout2D(0.25)(x)
        return x
        
    def initModel_S(self, dataset_name):
        assert dataset_name in ['CDnet', 'SBI', 'UCSD'], 'dataset_name must be either one in ["CDnet", "SBI", "UCSD"]]'
        assert len(self.img_shape)==3
        h, w, d = self.img_shape
        
        input_1 = Input(shape=(h, w, d), name='input')
        vgg_layer_output = self.VGG16(input_1)
        model = Model(inputs=input_1, outputs=vgg_layer_output, name='model')
        model.load_weights(self.vgg_weights_path, by_name=True)
        
        unfreeze_layers = ['block4_conv1','block4_conv2', 'block4_conv3']
        for layer in model.layers:
            if(layer.name not in unfreeze_layers):
                layer.trainable = False
                
        x = model.output
        
        if dataset_name=='CDnet':
            x1_ups = {'streetCornerAtNight':(0,1), 'tramStation':(1,0), 'turbulence2':(1,0)}
            for key, val in x1_ups.items():
                if self.scene==key:
                    # upscale by adding number of pixels to each dim.
                    x = MyUpSampling2D(size=(1,1), num_pixels=val)(x)
                    break
                
        x = self.FPM(x)
        x = self.transposedConv(x)
        
        if dataset_name=='CDnet':
            if(self.scene=='tramCrossroad_1fps'):
                x = MyUpSampling2D(size=(1,1), num_pixels=(2,0))(x)
            elif(self.scene=='bridgeEntry'):
                x = MyUpSampling2D(size=(1,1), num_pixels=(2,2))(x)
            elif(self.scene=='fluidHighway'):
                x = MyUpSampling2D(size=(1,1), num_pixels=(2,0))(x)
            elif(self.scene=='streetCornerAtNight'): 
                x = MyUpSampling2D(size=(1,1), num_pixels=(1,0))(x)
                x = Cropping2D(cropping=((0, 0),(0, 1)))(x)
            elif(self.scene=='tramStation'):  
                x = Cropping2D(cropping=((1, 0),(0, 0)))(x)
            elif(self.scene=='twoPositionPTZCam'):
                x = MyUpSampling2D(size=(1,1), num_pixels=(0,2))(x)
            elif(self.scene=='turbulence2'):
                x = Cropping2D(cropping=((1, 0),(0, 0)))(x)
                x = MyUpSampling2D(size=(1,1), num_pixels=(0,1))(x)
            elif(self.scene=='turbulence3'):
                x = MyUpSampling2D(size=(1,1), num_pixels=(2,0))(x)
        
        vision_model = Model(inputs=input_1, outputs=x, name='vision_model')
        opt = keras.optimizers.RMSprop(lr = self.lr, rho=0.9, epsilon=1e-08, decay=0.)
        
        # Since UCSD has no void label, we do not need to filter out
        if dataset_name == 'UCSD':
            c_loss = loss2
            c_acc = acc2
        else:
            c_loss = loss
            c_custom_f1 = custom_f1
            c_acc = acc
            
        vision_model.compile(loss=c_loss, optimizer=opt, metrics=[c_custom_f1, c_acc])
        return vision_model

In [14]:
"""
Created on Mon Aug  6 15:20:23 2018
@author: longang
"""

import keras
from keras import backend as K
from keras.engine.topology import Layer
from keras.utils import conv_utils
from keras.legacy import interfaces

# version 2.1.x has now base_layer class, so we need to import
if keras.__version__<'2.2':
    from keras.engine.topology import InputSpec
else:
    from keras.engine.base_layer import InputSpec

import tensorflow as tf
import numpy as np

class MyUpSampling2D(Layer):
    
    @interfaces.legacy_upsampling2d_support
    def __init__(self, size=(2, 2), num_pixels = (0, 0), data_format='channels_last', method_name='FgSegNet_M', **kwargs):
        super(MyUpSampling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.size = conv_utils.normalize_tuple(size, 2, 'size')
        self.input_spec = InputSpec(ndim=4)
        self.num_pixels = num_pixels
        self.method_name = method_name
        assert method_name in ['FgSegNet_M', 'FgSegNet_S', 'FgSegNet_v2'], 'Provided method_name is incorrect.'

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] + self.num_pixels[0] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] + self.num_pixels[1] if input_shape[2] is not None else None
            return (input_shape[0], height, width, input_shape[3])
        
        else:
            raise ValueError('Invalid data_format:', self.data_format)
        
    def call(self, inputs):
        return resize_images(inputs, self.size[0], self.size[1], self.data_format, self.num_pixels, self.method_name)

    def get_config(self):
        config = {'size': self.size,
                  'data_format': self.data_format,
                  'num_pixels': self.num_pixels}
        base_config = super(MyUpSampling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# keras func: tensorflow_backend
def resize_images(x, height_factor, width_factor, data_format, num_pixels=None, method_name=None):
    """Resizes the images contained in a 4D tensor.
    # Arguments
        x: Tensor or variable to resize.
        height_factor: Positive integer.
        width_factor: Positive integer.
        data_format: string, `"channels_last"`
    # Returns
        A tensor.
    # Raises
        ValueError: if `data_format` is neither `"channels_last"`.
    """
    
    if data_format == 'channels_last':
        original_shape = K.int_shape(x) # (None, 67, 90, 512)
        new_shape = tf.shape(x)[1:3] # (67, 90, 512)
        
        #print(new_shape.get_shape().as_list())
        new_shape *= tf.constant(np.array([height_factor, width_factor]).astype('int32'))
        if(num_pixels is not None):
           new_shape += tf.constant(np.array([num_pixels[0], num_pixels[1]]).astype('int32'))
        
        if method_name in ['FgSegNet_M', 'FgSegNet_S']:
            x = tf.compat.v1.image.resize_nearest_neighbor(x, new_shape)
        else: # FgSegNet_v2
            x = tf.image.resize_bilinear(x, new_shape)
        
        if(num_pixels is not None):
            x.set_shape((None, original_shape[1] * height_factor + num_pixels[0] if original_shape[1] is not None else None,
                     original_shape[2] * width_factor + num_pixels[1] if original_shape[2] is not None else None, None))
        else:
            x.set_shape((None, original_shape[1] * height_factor if original_shape[1] is not None else None,
                     original_shape[2] * width_factor if original_shape[2] is not None else None, None))
        return x
    else:
        raise ValueError('Invalid data_format:', data_format)

In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 29 22:22:22 2017
@author: longang
"""

#get_ipython().magic(u'load_ext autoreload')
#get_ipython().magic(u'autoreload 2')

import numpy as np
import tensorflow as tf
import random as rn
import os, sys
import time

#print(tf.test.is_gpu_available())
#print(tf.test.is_built_with_cuda())
#print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))
#print(tf.__version__)

# set current working directory
#cur_dir = os.getcwd()
#os.chdir(cur_dir)
#sys.path.append(cur_dir)

# =============================================================================
#  For reprodocable results
# =============================================================================
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
#session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#session_conf.gpu_options.per_process_gpu_memory_fraction = 0.6
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#from keras import backend as K
from keras import backend as K
import PIL
from PIL import Image
#from sklearn.preprocessing import LabelEncoder
import PIL.Image

import keras, glob
from keras.preprocessing import image as kImage
from skimage.transform import pyramid_gaussian
from sklearn.utils import compute_class_weight
#from FgSegNet_M_S_module import FgSegNet_M_S_module
from keras.utils.data_utils import get_file


#tf.config.experimental.set_visible_devices([], 'GPU')

#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#print("physical_devices-------------", len(physical_devices))
##tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.set.random_seed(1234)
#sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
#K.set_session(sess)

#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.6  # 0.6 sometimes works better for folks
#keras.backend.tensorflow_backend.set_session(tf.Session(config=config))



# alert the user
#if keras.__version__!= '2.0.6' or tf.__version__!='1.1.0' or sys.version_info[0]<3:
#    print('We implemented using [keras v2.0.6, tensorflow-gpu v1.1.0, python v3.6.3], other versions than these may cause errors somehow!\n')

# =============================================================================
# Few frames, load into memory directly
# =============================================================================
def generateData(train_dir, dataset_dir, scene, method_name):
    assert method_name in ['FgSegNet_M', 'FgSegNet_S'], 'method_name is incorrect'
    
    void_label = -1.
    
    # Given ground-truths, load training frames
    # ground-truths end with '*.png'
    # training frames end with '*.jpg'
    
    # given ground-truths, load inputs  
    Y_list = glob.glob(os.path.join(train_dir, '*.png'))
    X_list= glob.glob(os.path.join(dataset_dir, 'input','*.jpg'))


    if len(Y_list)<=0 or len(X_list)<=0:
        raise ValueError('System cannot find the dataset path or ground-truth path. Please give the correct path.')
        
    X_list_temp = []
    for i in range(len(Y_list)):
        Y_name = os.path.basename(Y_list[i])
        Y_name = Y_name.split('.')[0]
        Y_name = Y_name.split('gt')[1]
        for j in range(len(X_list)):
            X_name = os.path.basename(X_list[j])
            X_name = X_name.split('.')[0]
            X_name = X_name.split('in')[1]
            if (Y_name == X_name):
                X_list_temp.append(X_list[j])
                break
            
    X_list = X_list_temp
    
    if len(X_list)!=len(Y_list):
        raise ValueError('The number of X_list and Y_list must be equal.')
        
    # X must be corresponded to Y
    X_list = sorted(X_list)
    Y_list = sorted(Y_list)

    # load training data
    X = []
    Y = []
    for i in range(len(X_list)):
        x = kImage.load_img(X_list[i])
        x = kImage.img_to_array(x)
        X.append(x)
        
        x = kImage.load_img(Y_list[i], grayscale = True)
        x = kImage.img_to_array(x)
        shape = x.shape
        x /= 255.0
        x = x.reshape(-1)
        idx = np.where(np.logical_and(x>0.25, x<0.8))[0] # find non-ROI
        if (len(idx)>0):
            x[idx] = void_label
        x = x.reshape(shape)
        x = np.floor(x)
        Y.append(x)
        
    X = np.asarray(X)
    Y = np.asarray(Y)
        
    # We do not consider temporal data
    idx = list(range(X.shape[0]))
    np.random.shuffle(idx)
    np.random.shuffle(idx)
    X = X[idx]
    Y = Y[idx]
    
    if method_name=='FgSegNet_M':
        # Image Pyramid
        scale2 = []
        scale3 = []
        for i in range(0, X.shape[0]):
           pyramid = tuple(pyramid_gaussian(X[i]/255., max_layer=2, downscale=2))
           scale2.append(pyramid[1]*255.) # 2nd scale
           scale3.append(pyramid[2]*255.) # 3rd scale
           del pyramid
           
        scale2 = np.asarray(scale2)
        scale3 = np.asarray(scale3)

    # compute class weights
    cls_weight_list = []
    for i in range(Y.shape[0]):
        y = Y[i].reshape(-1)
        idx = np.where(y!=void_label)[0]
        if(len(idx)>0):
            y = y[idx]
        lb = np.unique(y) #  0., 1
        cls_weight = compute_class_weight('balanced', lb , y)
        class_0 = cls_weight[0]
        class_1 = cls_weight[1] if len(lb)>1 else 1.0
        
        cls_weight_dict = {0:class_0, 1: class_1}
        cls_weight_list.append(cls_weight_dict)

    cls_weight_list = np.asarray(cls_weight_list)
    
    if method_name=='FgSegNet_M':
        return [X, scale2, scale3, Y, cls_weight_list]
    else:
        return [X,Y,cls_weight_list]
    
def train(results, scene, mdl_path, vgg_weights_path, method_name):
    assert method_name in ['FgSegNet_M', 'FgSegNet_S'], 'method_name is incorrect'
    
    img_shape = results[0][0].shape # (height, width, channel)
    model = FgSegNet_M_S_module(lr, reg, img_shape, scene, vgg_weights_path)
    
    if method_name=='FgSegNet_M':
        model = model.initModel_M('CDnet')
    else:
        model = model.initModel_S('CDnet')
    
    # make sure that training input shape equals to model output
    input_shape = (img_shape[0], img_shape[1])
    output_shape = (model.output._keras_shape[1], model.output._keras_shape[2])
    assert input_shape==output_shape, 'Given input shape:' + str(input_shape) + ', but your model outputs shape:' + str(output_shape) 

    chk = keras.callbacks.ModelCheckpoint(mdl_path, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    redu = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=reduce_factor, patience=num_patience, verbose=1, mode='auto')
    early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=0, mode='auto')

    if method_name=='FgSegNet_M':
        model.fit([results[0], results[1], results[2]], results[3], validation_split=val_split, epochs=max_epochs, batch_size=batch_size, 
                           callbacks=[redu, chk], verbose=1, class_weight=results[4], shuffle = True)
    else:
        # maybe we can use early stopping instead for FgSegNet_S, and also set max epochs to 100 or 110
        t0=time.time()
        model.fit(results[0], results[1], validation_split=val_split, epochs=max_epochs, batch_size=batch_size, 
              callbacks=[redu, early], verbose=1, class_weight=results[2], shuffle = True)
        print ("training time:",category, ">", scene,  round(time.time()-t0, 3), "s")
        model.save(mdl_path)
        
    del model, results, chk, redu, early


# =============================================================================
# Main func
# =============================================================================
dataset = {
            'baseline':['highway'],
            'cameraJitter':['badminton'],
            'badWeather':['skating'],
            'dynamicBackground':['boats'],
            'intermittentObjectMotion':['abandonedBox'],
            'lowFramerate':['port_0_17fps'],
            'nightVideos':['bridgeEntry'],
            'PTZ':['continuousPan'],
            'shadow':['backdoor'],
            'thermal':['corridor'],
            'turbulence':['turbulence0']
}

# =============================================================================

method_name = 'FgSegNet_S' # either <FgSegNet_M> or <FgSegNet_S>, default FgSegNet_M

num_frames = 50 # either 50 or 200 frames, default 50 frames

reduce_factor = 0.1
num_patience = 6
lr = 1e-4
reg=5e-4
max_epochs = 50 
val_split = 0.2
batch_size = 1
# =============================================================================

# Example: (free to modify)

# FgSegNet/FgSegNet/FgSegNet_M_S_CDnet.py
# FgSegNet/FgSegNet/FgSegNet_M_S_SBI.py
# FgSegNet/FgSegNet/FgSegNet_M_S_UCSD.py
# FgSegNet/FgSegNet/FgSegNet_M_S_module.py

# FgSegNet/FgSegNet_dataset2014/...
# FgSegNet/CDnet2014_dataset/...


assert num_frames in [50,200], 'Incorrect number of frames'
main_dir = os.path.join('/content/drive/MyDrive/FgSegNet', method_name)
main_mdl_dir = os.path.join(main_dir, 'CDnet', 'models' + str(num_frames))
vgg_weights_path = '/content/drive/MyDrive/FgSegNet/FgSegNet/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
if not os.path.exists(vgg_weights_path):
    # keras func
    WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    vgg_weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                WEIGHTS_PATH_NO_TOP, cache_subdir='models',
                                file_hash='6d6bbae143d832006294945121d1f1fc')

print('*** Current method >>> ' + method_name + '\n')
for category, scene_list in dataset.items():
    
    mdl_dir = os.path.join(main_mdl_dir, category)
    if not os.path.exists(mdl_dir):
        os.makedirs(mdl_dir)
        
    for scene in scene_list:
        print ('Training ->>> ' + category + ' / ' + scene)
        
        # training frame path and dataset2014 path
        train_dir = os.path.join('/content/drive/MyDrive/FgSegNet', 'FgSegNet_dataset2014', category, scene + str(num_frames))
        dataset_dir = os.path.join('/content/drive/MyDrive/FgSegNet', 'CDnet2014_dataset', category, scene)
        results = generateData(train_dir, dataset_dir, scene, method_name)
        
        mdl_path = os.path.join(mdl_dir, 'mdl_' + scene + '.h5')
        train(results, scene, mdl_path, vgg_weights_path, method_name)
        del results

*** Current method >>> FgSegNet_S

Training ->>> baseline / highway
Train on 40 samples, validate on 10 samples
Epoch 1/50
40/40 [==============================] - 6s 155ms/step - loss: 0.4558 - custom_f1: 0.0113 - acc: 0.8720 - val_loss: 0.2759 - val_custom_f1: 0.0000e+00 - val_acc: 0.8999
Epoch 2/50
40/40 [==============================] - 2s 49ms/step - loss: 0.2636 - custom_f1: 0.4657 - acc: 0.9363 - val_loss: 0.2438 - val_custom_f1: 0.9495 - val_acc: 0.9904
Epoch 3/50
40/40 [==============================] - 2s 49ms/step - loss: 0.2281 - custom_f1: 0.9367 - acc: 0.9873 - val_loss: 0.1758 - val_custom_f1: 0.9780 - val_acc: 0.9956
Epoch 4/50
40/40 [==============================] - 2s 49ms/step - loss: 0.1557 - custom_f1: 0.9842 - acc: 0.9966 - val_loss: 0.1507 - val_custom_f1: 0.9751 - val_acc: 0.9949
Epoch 5/50
40/40 [==============================] - 2s 49ms/step - loss: 0.1408 - custom_f1: 0.9876 - acc: 0.9974 - val_loss: 0.1370 - val_custom_f1: 0.9833 - val_acc: 0.9965
Epoch 6/